In [1]:
import dask.dataframe as dd
import gc
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import numpy as np

# Cargar el DataFrame desde el archivo Parquet utilizando Dask
data_dd = dd.read_parquet(r'C:\Users\HOME\OneDrive - Universidad Nacional de Colombia\maestria_big_data\clases\TFM\codigo_TFM\data.parquet', engine='pyarrow')

# Convertir los datos de Dask a pandas para el preprocesamiento
data_pd = data_dd.compute()
X = data_pd['susceptibilidad'].values.reshape(-1, 1)
y = data_pd['inventario'].values

# Liberar memoria
del data_dd, data_pd
gc.collect()

# Dividir los datos en conjuntos de entrenamiento, validación y prueba
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Función para crear, entrenar y evaluar el modelo con diferentes pesos de clases
def train_and_evaluate(class_weight):
    # Construir el modelo secuencial
    model = Sequential([
        Dense(50, activation='relu', input_shape=(1,)),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    # Compilar el modelo
    model.compile(optimizer=Adam(learning_rate=0.001), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    
    # Configurar EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    # Entrenar el modelo con los datos de entrenamiento y validación
    model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=100, 
        batch_size=32, 
        verbose=0,
        callbacks=[early_stopping],
        class_weight=class_weight
    )
    
    # Evaluar el modelo en el conjunto de validación
    val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
    y_val_pred = model.predict(X_val).round()
    roc_auc = roc_auc_score(y_val, y_val_pred)
    
    return val_accuracy, roc_auc, model

# Paso 2: Probar diferentes pesos de clase
weights = [{0: 1, 1: w} for w in range(1, 51)]  # Probar pesos de 1 a 50 para la clase minoritaria
results = []

for weight in weights:
    val_accuracy, roc_auc, model = train_and_evaluate(weight)
    results.append((weight, val_accuracy, roc_auc))
    print(f'Pesos: {weight}, Validación Accuracy: {val_accuracy}, ROC AUC: {roc_auc}')


c:\Users\HOME\anaconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 610us/step
Pesos: {0: 1, 1: 1}, Validación Accuracy: 0.7455099821090698, ROC AUC: 0.5


c:\Users\HOME\anaconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 663us/step
Pesos: {0: 1, 1: 2}, Validación Accuracy: 0.7455099821090698, ROC AUC: 0.5


c:\Users\HOME\anaconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 610us/step
Pesos: {0: 1, 1: 48}, Validación Accuracy: 0.2544900178909302, ROC AUC: 0.5


c:\Users\HOME\anaconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 600us/step
Pesos: {0: 1, 1: 49}, Validación Accuracy: 0.2544900178909302, ROC AUC: 0.5


c:\Users\HOME\anaconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 466us/step
Pesos: {0: 1, 1: 50}, Validación Accuracy: 0.2544900178909302, ROC AUC: 0.5


In [2]:
# Paso 3: Seleccionar el mejor modelo basado en ROC AUC
best_weight = max(results, key=lambda x: x[2])  # Seleccionar por mejor ROC AUC
print(f'Mejor configuración de pesos: {best_weight[0]}, Accuracy: {best_weight[1]}, ROC AUC: {best_weight[2]}')

# Reentrenar el mejor modelo completo con los datos de entrenamiento y validación
best_model = Sequential([
    Dense(50, activation='relu', input_shape=(1,)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

best_model.compile(optimizer=Adam(learning_rate=0.001), 
                   loss='binary_crossentropy', 
                   metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

best_model.fit(
    np.concatenate((X_train, X_val)), 
    np.concatenate((y_train, y_val)),
    epochs=100, 
    batch_size=32, 
    verbose=1,
    callbacks=[early_stopping],
    class_weight=best_weight[0]
)




Mejor configuración de pesos: {0: 1, 1: 25}, Accuracy: 0.2575398087501526, ROC AUC: 0.5007299358431183
Epoch 1/100


c:\Users\HOME\anaconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


523/523 ━━━━━━━━━━━━━━━━━━━━ 1s 492us/step - accuracy: 0.3216 - loss: 85.6259
Epoch 2/100
303/523 ━━━━━━━━━━━━━━━━━━━━ 0s 510us/step - accuracy: 0.3024 - loss: 47.8522

c:\Users\HOME\anaconda3\lib\site-packages\keras\src\callbacks\early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


523/523 ━━━━━━━━━━━━━━━━━━━━ 0s 508us/step - accuracy: 0.3051 - loss: 44.8261
Epoch 3/100
523/523 ━━━━━━━━━━━━━━━━━━━━ 0s 495us/step - accuracy: 0.3115 - loss: 26.3444
Epoch 4/100
523/523 ━━━━━━━━━━━━━━━━━━━━ 0s 482us/step - accuracy: 0.3049 - loss: 11.2205
Epoch 5/100
523/523 ━━━━━━━━━━━━━━━━━━━━ 0s 485us/step - accuracy: 0.2798 - loss: 4.6968
Epoch 6/100
523/523 ━━━━━━━━━━━━━━━━━━━━ 0s 511us/step - accuracy: 0.2522 - loss: 3.4234
Epoch 7/100
523/523 ━━━━━━━━━━━━━━━━━━━━ 0s 507us/step - accuracy: 0.2525 - loss: 2.7466
Epoch 8/100
523/523 ━━━━━━━━━━━━━━━━━━━━ 0s 497us/step - accuracy: 0.2495 - loss: 2.6461
Epoch 9/100
523/523 ━━━━━━━━━━━━━━━━━━━━ 0s 481us/step - accuracy: 0.2593 - loss: 2.8888
Epoch 10/100
523/523 ━━━━━━━━━━━━━━━━━━━━ 0s 477us/step - accuracy: 0.2525 - loss: 2.9192
Epoch 11/100
523/523 ━━━━━━━━━━━━━━━━━━━━ 0s 511us/step - accuracy: 0.2535 - loss: 2.5192
Epoch 12/100
523/523 ━━━━━━━━━━━━━━━━━━━━ 0s 507us/step - accuracy: 0.2461 - loss: 2.6272
Epoch 13/100
523/523 ━━━━━━

c:\Users\HOME\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\HOME\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\HOME\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [3]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, verbose=0)
print(f'Loss en el conjunto de prueba: {test_loss}')
print(f'Accuracy en el conjunto de prueba: {test_accuracy}')



Loss en el conjunto de prueba: 1.714834451675415
Accuracy en el conjunto de prueba: 0.24695122241973877
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 387us/step
[[   0 2223]
 [   0  729]]
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      2223
         1.0       0.25      1.00      0.40       729

    accuracy                           0.25      2952
   macro avg       0.12      0.50      0.20      2952
weighted avg       0.06      0.25      0.10      2952



c:\Users\HOME\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\HOME\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\HOME\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [4]:
# Predecir y mostrar métricas adicionales en el conjunto de prueba
y_test_pred = best_model.predict(X_test).round()
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 397us/step
[[   0 2223]
 [   0  729]]
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      2223
         1.0       0.25      1.00      0.40       729

    accuracy                           0.25      2952
   macro avg       0.12      0.50      0.20      2952
weighted avg       0.06      0.25      0.10      2952



c:\Users\HOME\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\HOME\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\HOME\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
